## Import modules and functions

In [1]:
# -*- coding: utf-8 -*-
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm

import matplotlib.pyplot as plt
import pylab as pl
import numpy as np
import glob
import re
import os
import math

import pylab as pl
import matplotlib
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from matplotlib import rcParams
from scipy.signal import savgol_filter

from matplotlib.gridspec    import GridSpec
import import_ipynb

import Module_Diagnostics as MD
import numpy as np
from tempfile import mkdtemp
import os.path as path
import sys

from pathlib import Path
import json

#Debugger. For some reason, using it inside a function works well. Otherwise...
from IPython.core.debugger import set_trace
#exemple: 
# def debug():
#     set_trace()
    
#     `code_to_debug`
    
#     return

# debug()

importing Jupyter notebook from Module_Diagnostics.ipynb
testing compute_RMS(...):
This should be close to 1: 1.0094744220983753
This should be close to 0: 0.0


## Plot parameters

In [2]:
# %matplotlib notebook
rcParams["figure.figsize"] = [9.4, 4.8]
# matplotlib.use('nbagg') #_comment this line if you don't need to interact with plots (zoom, translations, savings...)

## Choose run and time for analysis

In [3]:
run_name = 'RUN_NAME'

loop = False #LOOP #(LOOP is a boolean)
time = 231 #TIME
time = '%05d' % time    # Change the time to string format, needed by functions

### Only for use:

#if working on lx-moissard
Cluster = 'Occ/'
run_name = '20_08_18_new_big_one_0'
# run_name = '20_09_07_start_big_one_2_bis'
# run_name = "20_05_18_event_only"
filepath = '/data/Lathys/Visualisation/' + Cluster + run_name + '/ncfiles/'

#if working on occigene
# filepath = '../ncfiles/'

try:
    date = re.search('Magw_(.+?)_t', glob.glob(filepath+'Magw*_t'+time+'.nc')[0]).group(1)
except (IndexError, AttributeError): 
    sys.exit(f"""time_dump {time} does not appear to have data.
             Let us go to the next time_dump""")

print(f'date of the simulation (DD_MM_YY): {date}')
print(f'time dump (in 1/omega_ci): {time}')

#This is used by the functions find_ip_shock(N, V) and find_mc_leading_edge(B)
metadata = {'t_shock_entrance' : 130,
            't_shock_exit'     : 240,
            't_MC_entrance'    : 130,
            't_MC_exit'        : 270}
#todo: autodefine t_collision? maybe from story_reader will be easier, as lines will cross on the multivariate plot

date of the simulation (DD_MM_YY): 18_08_20
time dump (in 1/omega_ci): 00230


In [4]:
# Prepare for plt.savefig
storing_directory = filepath + "../structure_images/"
path_png = Path(storing_directory)
time_label = f"t{time}"
if path_png.exists():
    pass
else:
    path_png.mkdir()

In [5]:
storing_directory_json = filepath + "../json_files/"

path_store_json = Path(storing_directory_json)

if not(path_store_json.exists()):
    os.system(f'mkdir {path_store_json}')

name = "sav_shock_understanding_" + run_name + ".json"
path_json = Path(storing_directory_json + name)

In [6]:
#The next line links directly to my PhD_Manuscript on Overleaf.com
#Saving plots there means that I do not need to worry about having the very last versions before
#starting to write about it in Overleaf, since they will always be up-to-date.

#IMPORTANT: for it to work, add the following line to ~/.bashrc
#export DROPBOX="/home/moissard/Dropbox" (to adapt to the particular machine)
dropbox = os.environ['DROPBOX']
dropbox_dir = dropbox + "/Apps/Overleaf/PhD_Manuscript/Images/LATHYS/"

# plt.savefig(dropbox_dir + '/' + saving_title + run_name + ".png",
#             transparent=True, bbox_inches='tight', pad_inches=0.1)

#If not working on the manuscript anymore, avoid changing images there by mistake
dropbox_dir = storing_directory

## Load data to large 3D arrays: N, (Bx, By, Bz), _etc._

#### Clear /tmp

In [7]:
print("""Clearing up /tmp/ ...
Indeed by using memmap, this code creates heavy temporary files""")
#Note: >/dev/null 2>&1 makes the system silent. Usually this command raises a lot of
# 'Action not permitted'. But that's fine. Nothing to debug here. tmp/ is full of
# files that should not be deleted, which are protected by root privileges.
os.system('rm -rf /tmp/* >/dev/null 2>&1')

Clearing up /tmp/ ...
Indeed by using memmap, this code creates heavy temporary files


256

### Upload B, n, E, T, V

In [8]:
## Load B and X, Y, Z

# There might be some MemoryError. In that case, follow the guide:
# https://stackoverflow.com/questions/60563936/memoryerror-unable-to-allocate-3-78-gib-for-an-array-with-shape-802-842-1502
Magw = MD.import_data_3D(filepath, date, time, 'Magw')

cwp = Magw['c_omegapi'][0]
gstep = Magw['gstep']
r_planet = Magw['r_planet'][0]

#Thomas Huret recommands shifting the axes so that the planet is indeed in (0,0,0)
#It has got something to do with lists starting at 1 instead of 0 in Fortran...?
X = np.array(np.around(Magw['x']))#+gstep[0]  
Y = np.array(np.around(Magw['y']))#+gstep[1]
Z = np.array(np.around(Magw['z']))#-gstep[1]

nx,  ny,  nz  = len(X), len(Y), len(Z)
# Location of the planet is defined in the .ncfiles as (x,y,z) = (0,0,0)
# Location of the planet is defined in the .ncfiles as (x,y,z) = (0,0,0)
nx0, ny0, nz0 = ( int(np.where(abs(X)==min(abs(X)))[0]),
                  int(np.where(abs(Y)==min(abs(Y)))[0]), 
                  int(np.where(abs(Z)==min(abs(Z)))[0])  )
            
# Use memmap to alleviate RAM
# This stores big arrays on the disk, but in a way that still allows for most
# operations available on an np.array
print("storing Magnetic field in a memmap")
file_Bx = path.join(mkdtemp(), 'Bx.dat')            
Bx = np.memmap(file_Bx, dtype='float32', mode='w+', shape=(nx,ny,nz))
Bx[:] = Magw['Bx']
file_By = path.join(mkdtemp(), 'By.dat')            
By = np.memmap(file_By, dtype='float32', mode='w+', shape=(nx,ny,nz))
By[:] = Magw['By']
file_Bz = path.join(mkdtemp(), 'Bz.dat')            
Bz = np.memmap(file_Bz, dtype='float32', mode='w+', shape=(nx,ny,nz))
Bz[:] = Magw['Bz']
print("deleting Magw to alleviate RAM")
del Magw
B = [Bx, By, Bz]

## Load E
# Electric field in mV/m
Elew = MD.import_data_3D(filepath, date, time, 'Elew')
print("storing Electric field in a memmap")
file_Ex = path.join(mkdtemp(), 'Ex.dat')            
Ex = np.memmap(file_Ex, dtype='float32', mode='w+', shape=(nx,ny,nz))
Ex[:] = Elew['Ex']*1e6
file_Ey = path.join(mkdtemp(), 'Ey.dat')            
Ey = np.memmap(file_Ey, dtype='float32', mode='w+', shape=(nx,ny,nz))
Ey[:] = Elew['Ey']*1e6
file_Ez = path.join(mkdtemp(), 'Ez.dat')            
Ez = np.memmap(file_Ez, dtype='float32', mode='w+', shape=(nx,ny,nz))
Ez[:] = Elew['Ez']*1e6
print("deleting Elew to alleviate RAM")
del Elew
E = [Ex, Ey, Ez]

## Load N, Vxyz, and T
Hsw = MD.import_data_3D(filepath, date, time, 'Hsw')
print("storing Plasma parameters in memmaps")
# Density in nb/cm^3
file_N = path.join(mkdtemp(), 'N.dat')            
N = np.memmap(file_N, dtype='float32', mode='w+', shape=(nx,ny,nz))
N[:] = Hsw['n']
# Velocity in km/s
file_Vx = path.join(mkdtemp(), 'Vx.dat')            
Vx = np.memmap(file_Vx, dtype='float32', mode='w+', shape=(nx,ny,nz))
Vx[:] = Hsw['Vx']
file_Vy = path.join(mkdtemp(), 'Vy.dat')            
Vy = np.memmap(file_Vy, dtype='float32', mode='w+', shape=(nx,ny,nz))
Vy[:] = Hsw['Vy']
file_Vz = path.join(mkdtemp(), 'Vz.dat')            
Vz = np.memmap(file_Vz, dtype='float32', mode='w+', shape=(nx,ny,nz))
Vz[:] = Hsw['Vz']
# Temperature in eV
file_T = path.join(mkdtemp(), 'T.dat')            
T = np.memmap(file_T, dtype='float32', mode='w+', shape=(nx,ny,nz))
T[:] = Hsw['T']
print("deleting Hsw to alleviate RAM")
del Hsw
V = [Vx, Vy, Vz]

Importing Magw 3D from /data/Lathys/Visualisation/Occ/20_08_18_new_big_one_0/ncfiles/
Reading Bx...
Reading By...
Reading Bz...
Close file and return...
storing Magnetic field in a memmap
deleting Magw to alleviate RAM
Importing Elew 3D from /data/Lathys/Visualisation/Occ/20_08_18_new_big_one_0/ncfiles/
Reading Ex...
Reading Ey...
Reading Ez...
Close file and return...
storing Electric field in a memmap
deleting Elew to alleviate RAM
Importing Hsw 3D from /data/Lathys/Visualisation/Occ/20_08_18_new_big_one_0/ncfiles/
Reading density...
Reading Ux...
Reading Uy...
Reading Uz...
Reading T...
Close file and return...
storing Plasma parameters in memmaps
deleting Hsw to alleviate RAM


In [11]:
np.shape(Bx)

(1502, 722, 662)

## Plot global, interactive view

In [37]:
xmin = min(X)
xmax = 3*abs(xmin)   #Most of the length in the x direction is used to give space for the sheath to form
                     #It is not really interesting
zoom = (xmin, xmax, min(Y), max(Y), min(Z), max(Z))
# zoom = (-100, 70, -200, 200, -200, 200)

In [ ]:
#Make plot interactive
%matplotlib notebook

In [ ]:
MD.plot_colormap(N[:,:,nz0], f'Density at {time} in (xy)', r'N (cm$^{-3}$)', 'xy',
                 loop = loop,
                 save_dir = storing_directory, t_label = time_label, zoom = zoom)

In [ ]:
position_shock_xy = plt.ginput(1, timeout = -1)

In [ ]:
MD.plot_colormap(N[:,ny0,:], f'Density at {time} in (xz)', r'N (cm$^{-3}$)', 'xz',
                 loop = loop,
                 save_dir = storing_directory, t_label = time_label, zoom = zoom)

In [ ]:
position_shock_xz = plt.ginput(1, timeout = -1)

## Generate relevant data

### Define cubes

In [ ]:
# Simple geometry assumption: shock normal is along x. This is not completely true.
# Should we do something about it? It's tough because we don't have that many grid cells to play with.

def cubes(position_shock, plan):
    
    '''Defines two cubes: one on the left and one on the right side of
       a point.'''
    
    distance = 10
    size_cubes = 4 # Goals and poles. This will be a 3 by 3 cube.
    
    x_shock = position_shock[0]
    if plan == 'xy':
        y_shock = position_shock[1]
        z_shock = 0
    if plan == 'xz':
        y_shock = 0
        z_shcok = position_shock[1]
    
    center_cube = x_shock, y_shock, z_shock

    cube_x_max = x_shock + size_cubes/2
    cube_x_min = x_shock - size_cubes/2
    cube_y_max = y_shock + size_cubes/2
    cube_y_min = y_shock - size_cubes/2
    cube_z_max = z_shock + size_cubes/2
    cube_z_min = z_shock - size_cubes/2

    ix_min_left = int(np.where(cube_x_min + distance <=X)[0][0])    
    ix_max_left = int(np.where(cube_x_max + distance <=X)[0][0])
    ix_min_left = int(np.where(cube_x_min + distance <=X)[0][0])    
    ix_max_left = int(np.where(cube_x_max + distance <=X)[0][0])
    iy_min = int(np.where(cube_y_min<=Y)[0][0])
    iy_max = int(np.where(cube_y_max<=Y)[0][0])
    iz_min = int(np.where(cube_z_min<=Z)[0][0])
    iz_max = int(np.where(cube_z_max<=Z)[0][0])
    
    cubes = {
             'left':  (ix_min_left , ix_max_left , iy_min, iy_max, iz_min, iz_max) ,
             'right': (ix_min_right, ix_max_right, iy_min, iy_max, iz_min, iz_max) ,
             }
    
    return cubes

def function_data_in_cube(data1, data2=None, function1=MD.identity, function_both=None, cube):
    
    '''
    Returns the mean value of data in the different cubes.
    If data is a vector (len(data)==3), then the function returns the mean value the norm(data[cubes])
    '''
    
                    
    ixmax, ixmin, iymax, iymin, izmax, izmin = cube                                                                      boxes[box]["coord_magnetopause"])
        
    slices = (slice(ixmin, ixmax), slice(iymin, iymax), slice(izmin, izmax))
        
    if (not(data2)):                       
        result = function1( data1, slices )
            
    else:
        result = function_both( function1( data1, slices ), [data2[0][slices],
                                                             data2[1][slices],
                                                             data2[2][slices]] )
    data_in_cube = []        
        if (len(result) == 3):
            data_in_boxes.append( (np.mean(result[0]), np.mean(result[1]), np.mean(result[2])) )
        else:
            data_in_boxes.append( np.mean(result) )
                  
        

    return data_in_cube

### Data in cubes

In [75]:
cubes_xy = cubes(position_shock_xy, 'xy')
cubes_xz = cubes(position_shock_xz, 'xz')

B_xy_left  = function_data_in_cube(B, function1=MD.norm, cubes_xy['left' ])
B_xy_right = function_data_in_cube(B, function1=MD.norm, cubes_xy['right'])

B_xz_left  = function_data_in_cube(B, function1=MD.norm, cubes_xz['left' ])
B_xz_right = function_data_in_cube(B, function1=MD.norm, cubes_xz['right'])

B_vec_xy_left  = function_data_in_cube(B, cubes_xy['left' ])
B_vec_xz_left  = function_data_in_cube(B, cubes_xz['left' ])

B_vec_xy_right = function_data_in_cube(B, cubes_xy['right'])
B_vec_xz_right = function_data_in_cube(B, cubes_xz['right'])

N_upstream, N_nose, N_yup, N_ydown, N_zup, N_zdown = MD.compute_data_in_cubes(N)

T_upstream, T_nose, T_yup, T_ydown, T_zup, T_zdown = MD.compute_data_in_cubes(T)

V_upstream, V_nose, V_yup, V_ydown, V_zup, V_zdown = MD.compute_data_in_cubes(V, function1=MD.norm)

Vx_upstream, Vx_nose, Vx_yup, Vx_ydown, Vx_zup, Vx_zdown = MD.compute_data_in_cubes(Vx)
Vy_upstream, Vy_nose, Vy_yup, Vy_ydown, Vy_zup, Vy_zdown = MD.compute_data_in_cubes(Vy)
Vz_upstream, Vz_nose, Vz_yup, Vz_ydown, Vz_zup, Vz_zdown = MD.compute_data_in_cubes(Vz)

print("calculating J")
J_upstream, J_nose, J_yup, J_ydown, J_zup, J_zdown = MD.compute_data_in_cubes(data1=B, function1=MD.J)

Jx_upstream, Jx_nose, Jx_yup, Jx_ydown, Jx_zup, Jx_zdown = MD.compute_data_in_cubes(B, function1=MD.Jx)
Jy_upstream, Jy_nose, Jy_yup, Jy_ydown, Jy_zup, Jy_zdown = MD.compute_data_in_cubes(B, function1=MD.Jy)
Jz_upstream, Jz_nose, Jz_yup, Jz_ydown, Jz_zup, Jz_zdown = MD.compute_data_in_cubes(B, function1=MD.Jz)

JE_upstream, JE_nose, JE_yup, JE_ydown, JE_zup, JE_zdown = MD.compute_data_in_cubes(data1=B, data2=E,
                                                                                    function1=MD.J,
                                                                                    function_both=MD.dot_product)


JxB_upstream, JxB_nose, JxB_yup, JxB_ydown, JxB_zup, JxB_zdown = MD.compute_data_in_cubes(data1=B, data2=B,
                                                                                          function1=MD.J,
                                                                                          function_both=MD.cross_product)

calculating J


In [ ]:
E_xy_left  = function_data_in_cube(E, function1=MD.norm, cubes_xy['left' ])
E_xy_right = function_data_in_cube(E, function1=MD.norm, cubes_xy['right'])

E_xz_left  = function_data_in_cube(E, function1=MD.norm, cubes_xz['left' ])
E_xz_right = function_data_in_cube(E, function1=MD.norm, cubes_xz['right'])

E_vec_xy_left  = function_data_in_cube(E, cubes_xy['left'])
E_vec_xz_left  = function_data_in_cube(E, cubes_xz['left'])

E_vec_xy_right = function_data_in_cube(E, cubes_xy['right'])
E_vec_xz_right = function_data_in_cube(E, cubes_xz['right'])

In [ ]:
V_xy_left  = function_data_in_cube(V, function1=MD.norm, cubes_xy['left' ])
V_xy_right = function_data_in_cube(V, function1=MD.norm, cubes_xy['right'])

V_xz_left  = function_data_in_cube(V, function1=MD.norm, cubes_xz['left' ])
V_xz_right = function_data_in_cube(V, function1=MD.norm, cubes_xz['right'])

V_vec_xy_left  = function_data_in_cube(V, cubes_xy['left'])
V_vec_xz_left  = function_data_in_cube(V, cubes_xz['left'])

V_vec_xy_right = function_data_in_cube(V, cubes_xy['right'])
V_vec_xz_right = function_data_in_cube(V, cubes_xz['right'])

In [ ]:
N_vec_xy_left  = function_data_in_cube(N, cubes_xy['left' ])
N_vec_xz_left  = function_data_in_cube(N, cubes_xz['left' ])

N_vec_xy_right = function_data_in_cube(N, cubes_xy['right'])
N_vec_xz_right = function_data_in_cube(N, cubes_xz['right'])

In [ ]:
T_vec_xy_left  = function_data_in_cube(T, cubes_xy['left' ])
T_vec_xz_left  = function_data_in_cube(T, cubes_xz['left' ])

T_vec_xy_right = function_data_in_cube(T, cubes_xy['right'])
T_vec_xz_right = function_data_in_cube(T, cubes_xz['right'])

In [ ]:
J_xy_left = function_data_in_cube(B, function1=MD.J, cubes_xy['left'])
J_xz_left = function_data_in_cube(B, function1=MD.J, cubes_xz['left'])

J_xy_right = function_data_in_cube(B, function1=MD.J, cubes_xy['right'])
J_xz_right = function_data_in_cube(B, function1=MD.J, cubes_xz['right'])

Jx_xy_left = function_data_in_cube(B, function1=MD.Jx, cubes_xy['left'])
Jx_xz_left = function_data_in_cube(B, function1=MD.Jx, cubes_xz['left'])

Jx_xy_right = function_data_in_cube(B, function1=MD.Jx, cubes_xy['right'])
Jx_xz_right = function_data_in_cube(B, function1=MD.Jx, cubes_xz['right'])

Jy_xy_left = function_data_in_cube(B, function1=MD.Jy, cubes_xy['left'])
Jy_xz_left = function_data_in_cube(B, function1=MD.Jy, cubes_xz['left'])

Jy_xy_right = function_data_in_cube(B, function1=MD.Jy, cubes_xy['right'])
Jy_xz_right = function_data_in_cube(B, function1=MD.Jy, cubes_xz['right'])

Jz_xy_left = function_data_in_cube(B, function1=MD.Jz, cubes_xy['left'])
Jz_xz_left = function_data_in_cube(B, function1=MD.Jz, cubes_xz['left'])

Jz_xy_right = function_data_in_cube(B, function1=MD.Jz, cubes_xy['right'])
Jz_xz_right = function_data_in_cube(B, function1=MD.Jz, cubes_xz['right'])

In [ ]:
#conversion to SI:
b = 1e-9
n = 1e6
v = 1e3
t = 11605
#put pressure in nPa
p = 1e9

In [77]:
Pmag_xy_left = function_data_in_cube(p*(b*MD.norm(B))**2/(2*MD.µ0), cubes_xy['left'])
Pmag_xz_left = function_data_in_cube(p*(b*MD.norm(B))**2/(2*MD.µ0), cubes_xz['left'])

Pmag_xy_right = function_data_in_cube(p*(b*MD.norm(B))**2/(2*MD.µ0), cubes_xy['right'])
Pmag_xz_right = function_data_in_cube(p*(b*MD.norm(B))**2/(2*MD.µ0), cubes_xz['right'])

In [ ]:
Pdyn_xy_left = function_data_in_cube(p*(1./2)*MD.mp*(n*N)*(v*MD.norm(V))**2, cubes_xy['left'])
Pdyn_xz_left = function_data_in_cube(p*(1./2)*MD.mp*(n*N)*(v*MD.norm(V))**2, cubes_xz['left'])

Pdyn_xy_right = function_data_in_cube(p*(1./2)*MD.mp*(n*N)*(v*MD.norm(V))**2, cubes_xy['right'])
Pdyn_xz_right = function_data_in_cube(p*(1./2)*MD.mp*(n*N)*(v*MD.norm(V))**2, cubes_xz['right'])

In [ ]:
Pth_xy_left = function_data_in_cube(p*MD.kB*(n*N)*(t*T), cubes_xy['left'])
Pth_xz_left = function_data_in_cube(p*MD.kB*(n*N)*(t*T), cubes_xz['left'])

Pth_xy_right = function_data_in_cube(p*MD.kB*(n*N)*(t*T), cubes_xy['right'])
Pth_xz_right = function_data_in_cube(p*MD.kB*(n*N)*(t*T), cubes_xz['right'])

In [ ]:
VA_xy_left = B_xy_left / (MD.µ0 * MD.mp * N_xy_left)

va_test_1 = VA_xy_left
va_test_2 = np.sqrt(Pmag_xy_left / (MD.mp * N_xy_left)
n = 3
assert isclose(va_test_1, va_test_2, abs_tol=10**-n)

In [ ]:
VA_xy_left = B_xy_left / (MD.µ0 * MD.mp * N_xy_left)
VA_xz_left = B_xy_left / (MD.µ0 * MD.mp * N_xz_left)

VA_xy_right = B_xy_right / (MD.µ0 * MD.mp * N_xy_right)
VA_xz_right = B_xy_right / (MD.µ0 * MD.mp * N_xz_right)

## Check Hypotheses

### V_shock higher in (xz) than (xy) $$v_s = \frac{n_1 v_1 - n_2 v_2}{n_1 - n_2}$$ 

In [ ]:
Vs_xy = (N_xy_left * V_xy_left - N_xy_right * V_xy_right) / (N_xy_left - N_xy_right)

In [ ]:
Vs_xz = (N_xz_left * V_xz_left - N_xz_right * V_xz_right) / (N_xz_left - N_xz_right)

In [ ]:
Vs_xz > Vs_xy

### Hypothesis: $$v_A^{xy} > v_A^{xz} \implies M_{A (shock)}^{xy} < M_{A (shock)}^{xz} \implies T_{down}^{xy} < T_{down}^{xz} $$

In [ ]:
VA_xy_right > VA_xz_right 

In [ ]:
M_Ashock_xy = Vs_xy / VA_xy_right
M_Ashock_xz = Vs_xz / VA_xz_right

In [ ]:
M_Ashock_xy < M_Ashock_xz

In [1]:
T_xy_left < T_xz_left

NameError: name 'T_xy_left' is not defined

## Store relevant data for the current time dump